In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [13]:
#путь к базе
path_to_db_local = '/datasets/telecomm_csi.db'

In [14]:
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
    
else:
    raise Exception('Файл с базой данных SQLite не найден!')

In [15]:
if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [25]:
#выгружаем данные из таблицы, столбцы с лояльностью и новыми и старыми пользователями делаем через оператор CASE
query = '''SELECT u.user_id, u.lt_day,
u.age,
CASE WHEN u.gender_segment = 0 THEN 'мужчина'
WHEN u.gender_segment = 1 THEN 'женщина'
END AS gender_segment,
u.os_name, u.cpe_type_name, u.nps_score,
CASE WHEN u.nps_score >=9 THEN 'сторонники'
     WHEN u.nps_score <= 6 THEN 'критики'
     ELSE 'нейтралы'
     END AS nps_group,
CASE WHEN u.lt_day <= 365 THEN 'новый пользователь'
     ELSE 'старый пользователь'
     END AS is_new,
loc.city, loc.country,
SUBSTR(age.title, 4, 5) AS age_segment,
SUBSTR(tr.title, 4, 5) AS traffic_sefment,
SUBSTR(lt.title, 4, 5) AS lifetime_segment
FROM user AS u
LEFT JOIN location AS loc ON u.location_id = loc.location_id
LEFT JOIN age_segment AS age ON u.age_gr_id = age.age_gr_id
LEFT JOIN traffic_segment AS tr ON u.tr_gr_id = tr.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id'''

In [26]:
data = pd.read_sql(query, engine)

In [27]:
data.head()

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,nps_score,nps_group,is_new,city,country,age_segment,traffic_sefment,lifetime_segment
0,A001A2,2320,45.0,женщина,ANDROID,SMARTPHONE,10,сторонники,старый пользователь,Уфа,Россия,45-54,1-5,36+
1,A001WF,2344,53.0,мужчина,ANDROID,SMARTPHONE,10,сторонники,старый пользователь,Киров,Россия,45-54,1-5,36+
2,A003Q7,467,57.0,мужчина,ANDROID,SMARTPHONE,10,сторонники,старый пользователь,Москва,Россия,55-64,20-25,13-24
3,A004TB,4190,44.0,женщина,IOS,SMARTPHONE,10,сторонники,старый пользователь,РостовнаДону,Россия,35-44,0.1-1,36+
4,A004XT,1163,24.0,мужчина,ANDROID,SMARTPHONE,10,сторонники,старый пользователь,Рязань,Россия,16-24,5-10,36+


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   age               501939 non-null  float64
 3   gender_segment    501192 non-null  object 
 4   os_name           502493 non-null  object 
 5   cpe_type_name     502493 non-null  object 
 6   nps_score         502493 non-null  int64  
 7   nps_group         502493 non-null  object 
 8   is_new            502493 non-null  object 
 9   city              502493 non-null  object 
 10  country           502493 non-null  object 
 11  age_segment       502493 non-null  object 
 12  traffic_sefment   502493 non-null  object 
 13  lifetime_segment  502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [10]:
data.to_csv('projects_nps.csv')

[Презентация](https://disk.yandex.ru/i/mJ4pCQ2J_KS9NQ)

[Дашборд](https://public.tableau.com/views/NPSproject_16806840901880/CategoryDashbord?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)